In [1]:
from selenium import webdriver
from time import sleep
import random
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import pandas as pd
import os

In [42]:
browser = webdriver.Chrome()

In [43]:
browser.get('https://www.amazon.eg/?language=en_AE')

In [28]:
choose_phones_button = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[4]/div[2]/div[2]/div/a[3]')
choose_phones_button.click()


In [29]:
see_more = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[3]/div[14]/div/div/div[5]/div/div/a/span')
see_more.click()

In [30]:
page_number = browser.find_element(By.XPATH, "//span[@class='s-pagination-item s-pagination-disabled']")

In [31]:
print(page_number.text)

74


In [32]:
products_link_list =[]

In [ ]:
for i in range(1, int(page_number.text)):
    sleep(1.7)# Wait 1.7 seconds to allow the page to load
    product_elements = browser.find_elements(By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in product_elements:
        product_link = product.get_attribute('href')
        products_link_list.append(product_link)
    
    # Try to find the next button, retry after 5 seconds if not found
    try:
        nextButton = browser.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        nextButton.click()
    except NoSuchElementException:
        sleep(5)  # Wait 5 seconds before retrying

In [40]:
print(products_link_list)

['https://www.amazon.eg/-/en/sspa/click?ie=UTF8&spc=MTo2NjQ1Njk4Nzc1MjE1NDUzOjE3Mjc3MTQ1NDg6c3BfYXRmX2Jyb3dzZTozMDAzNDMzNzY0MTE1MzI6OjA6Og&url=%2FSamsung-Phantom-Smartphone-Manufacturer-Warranty%2Fdp%2FB0BSLBMCB2%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.ESuG0wU7gFJe36ODf9jPQUgrNOkoa-N-NVzNgViLhYVTRyceLmLFYrZExzuEMtAwxUTXiCHTLs9vkfkxe7uzAyeyl4UxhaRij7rk0hTD_T2jJ1JCriW8qI1eITWR-5OikcbaI5fplM5BWrF9GDFJq84EF6DshUGcuPYTForbOXofgeqWeN_ZGu6HaOfaNRawRDhV-gM7bER5EFZdOIa3XddSQ1XDfmBHYVvJoTadR3q-u_zmdjfm2nSoRHXSr4FvPGdJvsIzR1MxNeZtZmV2P5yn34hilA1suhRUwaSy1Gs.GaP7khONxd5ja8DG8FZ7m0_-q3kvKhFWScMTW9N5I_Y%26dib_tag%3Dse%26qid%3D1727714548%26s%3Delectronics%26sr%3D1-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1', 'https://www.amazon.eg/-/en/Samsung-Galaxy-Dual-256GGB-Inches/dp/B0CQJZ76X9/ref=sr_1_2?dib=eyJ2IjoiMSJ9.ESuG0wU7gFJe36ODf9jPQUgrNOkoa-N-NVzNgViLhYVTRyceLmLFYrZExzuEMtAwxUTXiCHTLs9vkfkxe7uzAyeyl4UxhaRij7rk0hTD_T2jJ1JCriW8qI1eITWR-5OikcbaI5fplM5BWrF9GDFJq84EF6DshUGcuPYTForbOXofg

In [41]:
#function to extract data from a single product page using Selenium
def extract_phone_info(product_link, driver):
    """
Extracts detailed phone product information from a single product page using Selenium.

Args:
    product_link (str): The URL of the phone product page to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the product page.

Returns:
    dict: A dictionary containing the following extracted product details:
        - category (str or None): The category of the product (in this case, "smartphones").
        - listing_name (str or None): The title of the product listing.
        - thumbnail_link (str or None): The URL of the product's thumbnail image.
        - brand_name (str or None): The brand name of the phone.
        - operating_system (str or None): The operating system used by the phone.
        - cpu_model (str or None): The CPU model of the phone.
        - memory_storage_capacity (str or None): The storage capacity of the phone.
        - screen_size (str or None): The size of the phone's screen.
        - model_name (str or None): The model name of the phone.
        - wireless_provider (str or None): The wireless provider for the phone.
        - cellular_technology (str or None): The type of cellular technology (e.g., 4G, 5G).
        - connectivity_technology (str or None): The types of connectivity (e.g., Bluetooth, Wi-Fi).
        - color (str or None): The color of the phone.
        - about_this_item (str or None): Key product details or bullet points.
        - reviews (list of str or None): A list of reviews for the product.
        - rating (str or None): The product's average rating.
        - price_before_promo (str or None): The price before any promotional discount.
        - price_after_promo/current price (str or None): The current or promotional price.
        - num_purchases_last_month (str or None): Number of purchases made in the previous month.
        - available_colours (list of str or None): Available color options for the product.
        - ram_memory_installed (str or None): The amount of RAM installed in the phone.

Notes:
    - Uses a series of try-except blocks to handle potential missing data for each product detail.
"""

    # Navigate to the product page
    driver.get(product_link)

    # Dictionary to hold the data for each product
    product_data = {}

    
    try:
        # Category
        product_data['category'] = "smartphones"
    except NoSuchElementException:
        product_data['category'] = None


    try:
        # 0. listing name 
         product_data['listing_name'] = driver.find_element(By.XPATH, "//span[@id='productTitle' and contains(@class, 'a-size-large')]").text.strip()
    except NoSuchElementException:
         product_data['listing_name'] = None

    try:
        # 1. Phone Thumbnail Link
        product_data['thumbnail_link'] = driver.find_element(By.ID, 'landingImage').get_attribute('src')
    except NoSuchElementException:
        product_data['thumbnail_link'] = None

    try:
        # 2. Brand Name
        product_data['brand_name'] = driver.find_element(By.CSS_SELECTOR, 'td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['brand_name'] = None

    try:
        # 3. Operating System
        product_data['operating_system'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-operating_system td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['operating_system'] = None

    try:
        # 4. CPU Model
        product_data['cpu_model'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-cpu_model.family')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['cpu_model'] = None


    try:
        # 5. Memory Storage Capacity
        product_data['memory_storage_capacity'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-memory_storage_capacity td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['memory_storage_capacity'] = None

    try:
        # 6. Screen Size
        product_data['screen_size'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-display.size')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['screen_size'] = None


    try:
        # 7. Model Name
        product_data['model_name'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-model_name')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['model_name'] = None


    try:
        # 8. Wireless Provider
        product_data['wireless_provider'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-wireless_provider')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['wireless_provider'] = None


    try:
        # 9. Cellular Technology
        product_data['cellular_technology'] = driver.find_element(By.XPATH, "//th[contains(text(), 'Wireless communication technologies')]/following-sibling::td").text.strip()
    except NoSuchElementException:
        product_data['cellular_technology'] = None

    try:
        # 10. Connectivity Technology
        product_data['connectivity_technology'] = driver.find_element(By.XPATH, "//th[contains(text(), 'Connectivity technologies')]/following-sibling::td").text.strip()
    except NoSuchElementException:
        product_data['connectivity_technology'] = None


    try:
        # 11. Color
        product_data['color'] = driver.find_element(By.ID, 'color_name_0').get_attribute('title').replace("Click to select ", "")
    except NoSuchElementException:
        product_data['color'] = None

    try:
        # 12. About this item
        li_elements = driver.find_elements(By.XPATH, "//div[@id='feature-bullets']//ul/li")
        about_this_item = [li.text.strip() for li in li_elements]
        product_data['about_this_item'] = "\n".join(about_this_item) if about_this_item else None
    except NoSuchElementException:
        product_data['about_this_item'] = None

    try:
        # 13. Reviews
        reviews = [review.text.strip() for review in driver.find_elements(By.XPATH, "//div[@class='a-row a-spacing-small review-data']//span[@data-hook='review-body']//span[@class='cr-original-review-content']")]
        product_data['reviews'] = reviews if reviews else None
    except NoSuchElementException:
        product_data['reviews'] = None


    try:
        # 14. Rating
        product_data['rating'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[5]/div/span[1]/span/span[1]/a/span').text.strip()
    except NoSuchElementException:
        product_data['rating'] = None

    try:
        # Price Before Promo
        product_data['price_before_promo'] = (driver.find_element(By.CLASS_NAME, "a-price.a-text-price").text.strip() if driver.find_elements(By.CLASS_NAME, "a-price.a-text-price") else driver.find_element(By.XPATH, "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[12]/div/div/div[3]/div[1]/span[2]/span[2]/span[2]").text.strip())
    except NoSuchElementException:
        product_data['price_before_promo'] = None

    try:
        # 16. Price After Promo
        price_after_promo = driver.find_element(By.XPATH, "//span[@class='a-price-whole']").text.strip()
        product_data['price_after_promo/current price'] = price_after_promo
    except NoSuchElementException:
        product_data['price_after_promo/current price'] = None


    try:
        # 17. Number of Purchases Last Month
        product_data['num_purchases_last_month'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[10]/div/div/span/span[1]').text.strip()
    except NoSuchElementException:
        product_data['num_purchases_last_month'] = None

    try:
        # 18. Available Colours
        product_data['available_colours'] = [li.get_attribute('title').replace("Click to select ", "") for li in driver.find_elements(By.CSS_SELECTOR, 'li[id^=color_name_]')]
    except NoSuchElementException:
        product_data['available_colours'] = None

    try:
        # 19. RAM Memory Installed
        product_data['ram_memory_installed'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-ram_memory.installed_size')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['ram_memory_installed'] = None
    
    return product_data


In [ ]:
# Function to handle multiple products and store in DataFrame using a loop
def extract_all_phones(products_link_list, driver):
    """
Extracts data from multiple phone product pages and stores the results in a Pandas DataFrame.

Args:
    products_link_list (list of str): A list of URLs for the phone product pages.
    driver (webdriver): An instance of Selenium WebDriver used to interact with each product page.

Returns:
    pd.DataFrame: A Pandas DataFrame where each row represents the data for one product, 
                  collected by `extract_phone_info`.
    
Notes:
    - The function uses a loop to process each product link and appends the extracted data to a list.
    - `tqdm` is used to display the progress of the scraping process.
"""

    product_data_list = []

    # Loop through each product link
    for link in tqdm(products_link_list):
        product_data = extract_phone_info(link, driver)
        product_data_list.append(product_data)

    # Create DataFrame from the collected product data
    df = pd.DataFrame(product_data_list)
    return df

In [ ]:
df = extract_all_phones(products_link_list,browser)

In [ ]:
df.to_csv('amazon_egypt_phones.csv', index=False)

In [45]:
smartPhonesDf = df[~df['listing_name'].str.contains('case|accessory|band|accessories|headphones|جراب|غطاء|سماعة|ساعة', case=False, na=False)]


In [46]:
smartPhonesDf.to_csv('smartphones.csv', index=False)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794 entries, 0 to 1793
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   category                  1794 non-null   object
 1   listing_name              1794 non-null   object
 2   thumbnail_link            1794 non-null   object
 3   brand_name                1618 non-null   object
 4   operating_system          384 non-null    object
 5   cpu_model                 385 non-null    object
 6   memory_storage_capacity   391 non-null    object
 7   screen_size               389 non-null    object
 8   model_name                544 non-null    object
 9   wireless_provider         235 non-null    object
 10  cellular_technology       250 non-null    object
 11  connectivity_technology   246 non-null    object
 12  color                     236 non-null    object
 13  about_this_item           1731 non-null   object
 14  reviews                 

In [23]:
browser.quit()

In [24]:
browser = webdriver.Firefox()

In [25]:
browser.get('https://www.amazon.eg/?language=en_AE')


In [26]:
electronics_button = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[4]/div[2]/div[2]/div/a[4]')
electronics_button.click()

In [15]:
select_headphones = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[1]/div[2]/div/section/div/div/div[2]/div[2]/div/ol/li[4]/a/div[1]/div/img')
select_headphones.click()

In [16]:
see_more = browser.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[2]/div[3]/div[9]/div/div/div[5]/div/div/a/span")
see_more.click()

In [17]:
page_number = browser.find_element(By.XPATH, "//span[@class='s-pagination-item s-pagination-disabled']")
print(page_number.text)

139


In [18]:
headphones_link_list =[]

In [19]:
for i in range(1, int(page_number.text)):
    sleep(1.7)
    product_elements = browser.find_elements(By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in product_elements:
        product_link = product.get_attribute('href')
        headphones_link_list.append(product_link)
    
    # Try to find the next button, retry after 5 seconds if not found
    try:
        nextButton = browser.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        nextButton.click()
    except NoSuchElementException:
        sleep(5)  # Wait 5 seconds before retrying

In [38]:
def extract_headphone_info(product_link, driver):
    """
Extracts detailed product information from a headphone product page using Selenium.

Args:
    product_link (str): The URL of the headphone product page to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the product page.

Returns:
    dict: A dictionary containing the following extracted product details:
        - category (str or None): The product category (e.g., electronics, headphones).
        - listing_name (str or None): The title of the product listing.
        - thumbnail_link (str or None): The URL of the product's thumbnail image.
        - brand_name (str or None): The brand name of the headphones.
        - HeadPhone_formFactor (str or None): The form factor of the headphones (e.g., over-ear, in-ear).
        - model_name (str or None): The model name of the headphones.
        - impedance (str or None): The impedance specification of the headphones.
        - connectivity_technology (str or None): The connectivity type (e.g., Bluetooth, wired).
        - color (str or None): The color of the currently selected headphone model.
        - about_this_item (str or None): Key product details or bullet points.
        - reviews (list of str or None): A list of reviews for the product.
        - rating (str or None): The product's average rating.
        - price_before_promo (str or None): The price before any promotional discount.
        - price_after_promo/current price (str or None): The current or promotional price.
        - num_purchases_last_month (str or None): The number of purchases made in the previous month.
        - available_colours (list of str or None): Available color options for the product.

Notes:
    - If the "See more" button is present, it is clicked to reveal additional product information.
    - If a particular detail is not found on the page, the corresponding value in the dictionary is set to None.
"""
    # Navigate to the product page
    driver.get(product_link)

    # Dictionary to hold the data for each product
    product_data = {}

    
    try:
        # Category 
        product_data['category'] = driver.find_element(By.XPATH, "(//ul[@class='a-unordered-list a-horizontal a-size-small']//a[@class='a-link-normal a-color-tertiary'])[last()]").text.strip()
    except NoSuchElementException:
        product_data['category'] = None


    try:
        # 0. listing name 
         product_data['listing_name'] = driver.find_element(By.XPATH, "//span[@id='productTitle' and contains(@class, 'a-size-large')]").text.strip()
    except NoSuchElementException:
         product_data['listing_name'] = None

    try:
        # 1. headphone Thumbnail Link
        product_data['thumbnail_link'] = driver.find_element(By.ID, 'landingImage').get_attribute('src')
    except NoSuchElementException:
        product_data['thumbnail_link'] = None

    try:
        # 2. Brand Name
        product_data['brand_name'] = driver.find_element(By.CSS_SELECTOR, 'td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['brand_name'] = None

    try:
        # 3. Headphones Form Factor
        product_data['HeadPhone_formFactor'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-headphones_form_factor td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['HeadPhone_formFactor'] = None


    try:
        # 4. Model Name
        product_data['model_name'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-model_name')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['model_name'] = None


    try:
        # 5. Impedance
        product_data['impedance'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-impedance td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['impedance'] = None


    try:
        # 6. Connectivity Technology or Wireless Communication Technology
        product_data['connectivity_technology'] = driver.find_element(By.XPATH, 
            "//tr[contains(@class, 'po-connectivity_technology')]//td[2]//span[contains(@class, 'po-break-word')] | //tr[contains(@class, 'po-wireless_communication_technology')]//td[2]//span[contains(@class, 'po-break-word')]"
        ).text.strip()
    except NoSuchElementException:
        product_data['connectivity_technology'] = None



    try:
        # 7. available Colors
        product_data['color'] = driver.find_element(By.ID, 'color_name_0').get_attribute('title').replace("Click to select ", "")
    except NoSuchElementException:
        product_data['color'] = None

    try:
        # 8. About this item
        li_elements = driver.find_elements(By.XPATH, "//div[@id='feature-bullets']//ul/li")
        about_this_item = [li.text.strip() for li in li_elements]
        product_data['about_this_item'] = "\n".join(about_this_item) if about_this_item else None
    except NoSuchElementException:
        product_data['about_this_item'] = None

    try:
        # 9. Reviews
        reviews = [review.text.strip() for review in driver.find_elements(By.XPATH, "//div[@class='a-row a-spacing-small review-data']//span[@data-hook='review-body']//span[@class='cr-original-review-content']")]
        product_data['reviews'] = reviews if reviews else None
    except NoSuchElementException:
        product_data['reviews'] = None


    try:
        # 10. Rating
        product_data['rating'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[5]/div/span[1]/span/span[1]/a/span').text.strip()
    except NoSuchElementException:
        product_data['rating'] = None

    try:
        # 11. Price Before Promo
        product_data['price_before_promo'] = (driver.find_element(By.CLASS_NAME, "a-price.a-text-price").text.strip() if driver.find_elements(By.CLASS_NAME, "a-price.a-text-price") else driver.find_element(By.XPATH, "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[12]/div/div/div[3]/div[1]/span[2]/span[2]/span[2]").text.strip())
    except NoSuchElementException:
        product_data['price_before_promo'] = None


    try:
        # 12. Price After Promo
        price_after_promo = driver.find_element(By.XPATH, "//span[@class='a-price-whole']").text.strip()
        product_data['price_after_promo/current price'] = price_after_promo
    except NoSuchElementException:
        product_data['price_after_promo/current price'] = None


    try:
        # 13. Number of Purchases Last Month
        product_data['num_purchases_last_month'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[10]/div/div/span/span[1]').text.strip()
    except NoSuchElementException:
        product_data['num_purchases_last_month'] = None

    try:
        # 14. Available Colours
        product_data['available_colours'] = [li.get_attribute('title').replace("Click to select ", "") for li in driver.find_elements(By.CSS_SELECTOR, 'li[id^=color_name_]')]
    except NoSuchElementException:
        product_data['available_colours'] = None

    return product_data


In [ ]:
# Function to handle multiple products and store in DataFrame using a loop
def extract_all_headphones(products_link_list, driver):
    """
Extracts product data from a list of headphone product pages and stores it in a Pandas DataFrame.

Args:
    products_link_list (list of str): A list of URLs for the headphone product pages to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the product pages.

Returns:
    pd.DataFrame: A DataFrame containing the collected product data for each headphone.

Notes:
    - Calls the `extract_headphone_info` function for each product link in the list.
    - Displays a progress bar for the extraction process using `tqdm`.
"""

    product_data_list = []

    # Loop through each product link
    for link in tqdm(products_link_list):
        product_data = extract_headphone_info(link, driver)
        product_data_list.append(product_data)

    # Create DataFrame from the collected product data
    df = pd.DataFrame(product_data_list)
    return df


In [35]:
# Create a new array with the first 5 elements for testing purposes
first_5_links = headphones_link_list[:5]


In [ ]:
df = extract_all_headphones(headphones_link_list,browser)

In [43]:
# List of headphone names to filter the DataFrame
headphone_nicknames = [
    "Headphones", "Headset", "Earphones", "Earbuds", "In-Ear Monitors", 
    "Over-Ear Headphones", "On-Ear Headphones", "Neckband Earphones", 
    "Bone Conduction Headphones", "Wireless Headphones", "Bluetooth Headset", 
    "Hands-Free Headset", "True Wireless Earbuds", "Sports Headphones", 
    "Gaming Headset", "Noise-Canceling Headphones", "Open-Ear Headphones", 
    "Clip-On Earphones", "Behind-the-Neck Headphones", "Smart Headset"
]

# Filter the DataFrame to keep only rows containing any of the nicknames in 'listing_name'
filtered_df = df[df['listing_name'].str.contains('|'.join(headphone_nicknames), case=False, na=False)]

In [44]:
filtered_df.to_csv('headphones.csv', index=False)

In [7]:
electronics_button = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[4]/div[2]/div[2]/div/a[4]')
electronics_button.click()

In [8]:
mobile_phones_accessories = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[1]/div[2]/div/section/div/div/div[2]/div[2]/div/ol/li[1]/a/div[1]/div/img')
mobile_phones_accessories.click()

In [9]:
accessories = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[3]/div[2]/div/div/div/ul/li[1]/span/span/div/a')
accessories.click()

In [49]:
chargers_powerbanks = browser.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[2]/div[3]/div[1]/div/div/div/ul/li[8]/span/span/div/a/h3")
chargers_powerbanks.click()

In [50]:
see_more = browser.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[2]/div[3]/div[8]/div/div/div[5]/div/div/a/span")
see_more.click()

In [55]:
page_number = browser.find_element(By.XPATH, "//span[@class='s-pagination-item s-pagination-disabled']")
print(page_number.text)

85


In [56]:
charger_link_list =[]

In [39]:
for i in range(1, int(page_number.text)):
    sleep(1.7)
    product_elements = browser.find_elements(By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in product_elements:
        product_link = product.get_attribute('href')
        charger_link_list.append(product_link)
    
    # Try to find the next button, retry after 5 seconds if not found
    try:
        nextButton = browser.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        nextButton.click()
    except NoSuchElementException:
        sleep(5)  # Wait 5 seconds before retrying

In [59]:
def extract_charger_info(product_link, driver):
    """
Extracts detailed product information from a charger product page using Selenium.

Args:
    product_link (str): The URL of the charger product page to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the product page.

Returns:
    dict: A dictionary containing the following extracted product details:
        - category (str or None): The product category (e.g., electronics, chargers).
        - listing_name (str or None): The title of the product listing.
        - thumbnail_link (str or None): The URL of the product's thumbnail image.
        - brand_name (str or None): The brand name of the charger.
        - connector_type (str or None): The type of connector for the charger.
        - model_name (str or None): The model name of the charger.
        - special_features (str or None): Any special features listed for the charger.
        - color (str or None): The color of the charger.
        - battery_capacity (str or None): The battery capacity, if applicable.
        - about_this_item (str or None): Key product details or bullet points.
        - reviews (list of str or None): A list of reviews for the product.
        - rating (str or None): The product's average rating.
        - price_before_promo (str or None): The price before any promotional discount.
        - price_after_promo/current price (str or None): The current or promotional price.
        - num_purchases_last_month (str or None): Number of purchases made in the previous month.
        - available_colours (list of str or None): Available color options for the product.

Notes:
    - Uses a `sleep` of 2 seconds to ensure that the page elements are fully loaded before interacting with them.
    - If the "See more" button is present, it is clicked to reveal additional product information.
    - Missing data is handled using `None` to indicate that the specific element was not found.
"""

    # Navigate to the product page
    driver.get(product_link)

    # Dictionary to hold the data for each product
    product_data = {}
    sleep(2)

    try:
        # Locate the "See more" button
        show_more_button = driver.find_element(By.XPATH, "//div[@id='poToggleButton']//a[@role='button']")
        show_more_button.click()
    except NoSuchElementException:
        print("Show more button not found")

    try:
        # Category 
        product_data['category'] = driver.find_element(By.XPATH, "(//ul[@class='a-unordered-list a-horizontal a-size-small']//a[@class='a-link-normal a-color-tertiary'])[last()]").text.strip()
    except NoSuchElementException:
        product_data['category'] = None


    try:
        # 0. listing name 
         product_data['listing_name'] = driver.find_element(By.XPATH, "//span[@id='productTitle' and contains(@class, 'a-size-large')]").text.strip()
    except NoSuchElementException:
         product_data['listing_name'] = None

    try:
        # 1. headphone Thumbnail Link
        product_data['thumbnail_link'] = driver.find_element(By.ID, 'landingImage').get_attribute('src')
    except NoSuchElementException:
        product_data['thumbnail_link'] = None

    try:
        # 2. Brand Name
        product_data['brand_name'] = driver.find_element(By.CSS_SELECTOR, 'td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['brand_name'] = None

    try:
        # 3. Detecting Connector Type
        product_data['connector_type'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-connector_type td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['connector_type'] = None


    try:
        # 4. Model Name
        product_data['model_name'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-model_name')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['model_name'] = None


    try:
        # 5. Detecting Special Features
        product_data['special_features'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-special_feature td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['special_features'] = None

    try:
        # 6. available Colors
        product_data['color'] = driver.find_element(By.ID, 'color_name_0').get_attribute('title').replace("Click to select ", "")
    except NoSuchElementException:
        product_data['color'] = None

    try:
        # 7. Detecting Battery Capacity
        product_data['battery_capacity'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-battery\\.capacity td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['battery_capacity'] = None

    try:
        # 8. About this item
        li_elements = driver.find_elements(By.XPATH, "//div[@id='feature-bullets']//ul/li")
        about_this_item = [li.text.strip() for li in li_elements]
        product_data['about_this_item'] = "\n".join(about_this_item) if about_this_item else None
    except NoSuchElementException:
        product_data['about_this_item'] = None

    try:
        # 9. Reviews
        reviews = [review.text.strip() for review in driver.find_elements(By.XPATH, "//div[@class='a-row a-spacing-small review-data']//span[@data-hook='review-body']//span[@class='cr-original-review-content']")]
        product_data['reviews'] = reviews if reviews else None
    except NoSuchElementException:
        product_data['reviews'] = None


    try:
        # 10. Rating
        product_data['rating'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[5]/div/span[1]/span/span[1]/a/span').text.strip()
    except NoSuchElementException:
        product_data['rating'] = None

    try:
        # 11. Price Before Promo
        product_data['price_before_promo'] = (driver.find_element(By.CLASS_NAME, "a-price.a-text-price").text.strip() if driver.find_elements(By.CLASS_NAME, "a-price.a-text-price") else driver.find_element(By.XPATH, "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[12]/div/div/div[3]/div[1]/span[2]/span[2]/span[2]").text.strip())
    except NoSuchElementException:
        product_data['price_before_promo'] = None


    try:
        # 12. Price After Promo
        price_after_promo = driver.find_element(By.XPATH, "//span[@class='a-price-whole']").text.strip()
        product_data['price_after_promo/current price'] = price_after_promo
    except NoSuchElementException:
        product_data['price_after_promo/current price'] = None


    try:
        # 13. Number of Purchases Last Month
        product_data['num_purchases_last_month'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[10]/div/div/span/span[1]').text.strip()
    except NoSuchElementException:
        product_data['num_purchases_last_month'] = None

    try:
        # 14. Available Colours
        product_data['available_colours'] = [li.get_attribute('title').replace("Click to select ", "") for li in driver.find_elements(By.CSS_SELECTOR, 'li[id^=color_name_]')]
    except NoSuchElementException:
        product_data['available_colours'] = None

    return product_data


In [ ]:
# Function to handle multiple products and store in DataFrame using a loop
def extract_all_charger(products_link_list, driver):
    """
Extracts product information for a list of charger product links and compiles the data into a Pandas DataFrame.

Args:
    products_link_list (list): A list of URLs, each representing a charger product page to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the webpages.

Returns:
    pandas.DataFrame: A DataFrame containing the extracted product information for all chargers.
        Each row represents a product, with columns corresponding to the data extracted by the `extract_charger_info` function.

Notes:
    - Uses `tqdm` to display a progress bar while iterating through the product links.
    - Calls the `extract_charger_info` function to extract information from each product link.
"""

    product_data_list = []

    # Loop through each product link
    for link in tqdm(products_link_list):
        product_data = extract_charger_info(link, driver)
        product_data_list.append(product_data)

    # Create DataFrame from the collected product data
    df = pd.DataFrame(product_data_list)
    return df


In [61]:
first_5_links = charger_link_list[:5]

In [ ]:
df = extract_all_charger(charger_link_list,browser)

In [66]:
df.to_csv('chargers.csv', index=False)

In [48]:
electronics_button = browser.find_element(By.XPATH,'/html/body/div[1]/header/div/div[4]/div[2]/div[2]/div/a[4]')
electronics_button.click()

In [49]:
mobile_phones_accessories = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[1]/div[2]/div/section/div/div/div[2]/div[2]/div/ol/li[1]/a/div[1]/div/img')
mobile_phones_accessories.click()

In [50]:
accessories = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[3]/div[2]/div/div/div/ul/li[1]/span/span/div/a')
accessories.click()

In [10]:
cases = browser.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[3]/div[1]/div/div/div/ul/li[1]/span/span/div/a')
cases.click()

In [12]:
see_more = browser.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[2]/div[3]/div[5]/div/div/div[5]/div/div/a/span")
see_more.click()

In [13]:
page_number = browser.find_element(By.XPATH, "//span[@class='s-pagination-item s-pagination-disabled']")
print(page_number.text)

400


In [14]:
case_link_list =[]

In [15]:
for i in range(1, 120): #lowering number of pages to loop through to avoid memory errors
    sleep(1.7)
    product_elements = browser.find_elements(By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for product in product_elements:
        product_link = product.get_attribute('href')
        case_link_list.append(product_link)
    
    # Try to find the next button, retry after 5 seconds if not found
    try:
        nextButton = browser.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
        nextButton.click()
    except NoSuchElementException:
        sleep(5)  # Wait 5 seconds before retrying

In [29]:
# Example function to extract data from a single product page using Selenium
def extract_case_info(product_link, driver):
    """
    Extracts product information from a specific case product page using Selenium.

    Args:
        product_link (str): URL of the product page to scrape.
        driver (webdriver): An instance of Selenium WebDriver for interacting with the webpage.

    Returns:
        dict: A dictionary containing the extracted product information. Keys in the dictionary include:
            - 'category' (str): The category of the product.
            - 'listing_name' (str): The title or name of the product listing.
            - 'thumbnail_link' (str): The URL of the product's thumbnail image.
            - 'brand_name' (str): The brand of the product.
            - 'model_name' (str): The model name of the product.
            - 'special_features' (str): Any special features of the product.
            - 'compatible_models' (str): Models compatible with the product.
            - 'color' (str): Available color of the product.
            - 'material' (str): Material of the product.
            - 'about_this_item' (str): Product description or features.
            - 'reviews' (list): List of customer reviews.
            - 'rating' (str): Product rating (out of 5).
            - 'price_before_promo' (str): Price before any promotional discounts.
            - 'price_after_promo/current price' (str): Current price after promotional discounts.
            - 'num_purchases_last_month' (str): Number of purchases made in the last month.
            - 'available_colours' (list): List of available colors for the product.
            - 'embellishment_feature' (str): Embellishment feature of the product.
            - 'water_resistance_level' (str): Water resistance level of the product.
            - 'pattern' (str): The product's pattern.

    Exceptions:
        Handles NoSuchElementException if an element is not found on the webpage, and assigns None to the relevant field.
    """
    # Navigate to the product page
    driver.get(product_link)

    # Dictionary to hold the data for each product
    product_data = {}

    try:
        # Category 
        product_data['category'] = driver.find_element(By.XPATH, "(//ul[@class='a-unordered-list a-horizontal a-size-small']//a[@class='a-link-normal a-color-tertiary'])[last()]").text.strip()
    except NoSuchElementException:
        product_data['category'] = None


    try:
        # 0. listing name 
         product_data['listing_name'] = driver.find_element(By.XPATH, "//span[@id='productTitle' and contains(@class, 'a-size-large')]").text.strip()
    except NoSuchElementException:
         product_data['listing_name'] = None

    try:
        # 1. headphone Thumbnail Link
        product_data['thumbnail_link'] = driver.find_element(By.ID, 'landingImage').get_attribute('src')
    except NoSuchElementException:
        product_data['thumbnail_link'] = None

    try:
        # 2. Brand Name
        product_data['brand_name'] = driver.find_element(By.CSS_SELECTOR, 'td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['brand_name'] = None


    try:
        # 4. Model Name
        product_data['model_name'] = driver.find_element(By.XPATH, "//tr[contains(@class, 'po-model_name')]//td[2]//span[contains(@class, 'po-break-word')]").text.strip()
    except NoSuchElementException:
        product_data['model_name'] = None


    try:
        # 5. Detecting Special Features
        product_data['special_features'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-special_feature td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['special_features'] = None

    
    try:
        # 6. available Colors
        product_data['color'] = driver.find_element(By.ID, 'color_name_0').get_attribute('title').replace("Click to select ", "")
    except NoSuchElementException:
        product_data['color'] = None

    try:
        # 7. Detecting Material
        product_data['material'] = driver.find_element(By.CSS_SELECTOR, 'tr.po-material td.a-span9').text.strip()
    except NoSuchElementException:
        product_data['material'] = None

    try:
        # 8. About this item
        li_elements = driver.find_elements(By.XPATH, "//div[@id='feature-bullets']//ul/li")
        about_this_item = [li.text.strip() for li in li_elements]
        product_data['about_this_item'] = "\n".join(about_this_item) if about_this_item else None
    except NoSuchElementException:
        product_data['about_this_item'] = None

    try:
        # 9. Reviews
        reviews = [review.text.strip() for review in driver.find_elements(By.XPATH, "//div[@class='a-row a-spacing-small review-data']//span[@data-hook='review-body']//span[@class='cr-original-review-content']")]
        product_data['reviews'] = reviews if reviews else None
    except NoSuchElementException:
        product_data['reviews'] = None


    try:
        # 10. Rating
        product_data['rating'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[5]/div/span[1]/span/span[1]/a/span').text.strip()
    except NoSuchElementException:
        product_data['rating'] = None

    try:
        # 11. Price Before Promo
        product_data['price_before_promo'] = (driver.find_element(By.CLASS_NAME, "a-price.a-text-price").text.strip() if driver.find_elements(By.CLASS_NAME, "a-price.a-text-price") else driver.find_element(By.XPATH, "/html/body/div[2]/div/div[5]/div[3]/div[4]/div[12]/div/div/div[3]/div[1]/span[2]/span[2]/span[2]").text.strip())
    except NoSuchElementException:
        product_data['price_before_promo'] = None


    try:
        # 12. Price After Promo
        price_after_promo = driver.find_element(By.XPATH, "//span[@class='a-price-whole']").text.strip()
        product_data['price_after_promo/current price'] = price_after_promo
    except NoSuchElementException:
        product_data['price_after_promo/current price'] = None


    try:
        # 13. Number of Purchases Last Month
        product_data['num_purchases_last_month'] = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[10]/div/div/span/span[1]').text.strip()
    except NoSuchElementException:
        product_data['num_purchases_last_month'] = None

    try:
        # 14. Available Colours
        product_data['available_colours'] = [li.get_attribute('title').replace("Click to select ", "") for li in driver.find_elements(By.CSS_SELECTOR, 'li[id^=color_name_]')]
    except NoSuchElementException:
        product_data['available_colours'] = None


    
        
    return product_data
        

In [ ]:
def extract_all_cases(products_link_list, driver):
    """
Extracts product information for a list of case product links and compiles the data into a Pandas DataFrame.

Args:
    products_link_list (list): A list of URLs, each representing a product page to scrape.
    driver (webdriver): An instance of Selenium WebDriver for interacting with the webpages.

Returns:
    pandas.DataFrame: A DataFrame containing the extracted product information for all cases.
        Each row represents a product, with columns corresponding to the data extracted by the `extract_case_info` function.

Notes:
    - Uses `tqdm` to display a progress bar as it loops through the product links.
    - Calls the `extract_case_info` function to extract information from each product link.
"""

    product_data_list = []

    # Loop through each product link
    for link in tqdm(products_link_list):
        product_data = extract_case_info(link, driver)
        product_data_list.append(product_data)

    # Create DataFrame from the collected product data
    df = pd.DataFrame(product_data_list)
    return df

In [17]:
test_link_cases = case_link_list[:5]

In [33]:
df = extract_all_cases(case_link_list,browser)

100%|██████████| 4258/4258 [3:10:32<00:00,  2.68s/it]  


In [34]:
df

,category,listing_name,thumbnail_link,brand_name,model_name,special_features,compatible_models,color,material,about_this_item,reviews,rating,price_before_promo,price_after_promo/current price,num_purchases_last_month,available_colours,embellishment_feature,water_resistance_level,pattern
0,Basic Cases,Luxury Leather iPhone Cases - Luxury Phone Cas...,https://m.media-amazon.com/images/I/71Pl-v99up...,IZONE,None,None,Mobiles,Black,Leather,PREMIUM APPLE LEATHER IPHONE CASE: Crafted fro...,None,None,None,410,None,"[Black, Blue, Brown, Green, Grey]",,,
1,Bumpers,"ELMO3EZZ Case for Samsung Galaxy S24, Transpar...",https://m.media-amazon.com/images/I/61VB8y9xXY...,ELMO3EZZ,None,,Samsung Galaxy S24,None,Silicone,1,None,None,None,130,None,[],,None,Galaxy
2,Flip Cases,For Samsung Z Fold 4 Case Mirror Mobile Phone ...,https://m.media-amazon.com/images/I/61bzDiD6IT...,Gotivi,None,None,"z fold 4 case, samsung galaxy fold 4 case, sam...",None,Plastic,Case for Samsung Galaxy Z Fold 4 case: case fo...,[Passt überhaupt nicht mit den Produktfotos zu...,None,EGP199.00,159,None,[],,,
3,Basic Cases,Nokia 106 TA-1564 DS NENA1 Dual SIM 48MB Ram+1...,https://m.media-amazon.com/images/I/515X1z3CmM...,Nokia,,None,None,None,None,"Feature Phone\nMP3 Player ,Micro SD Support /W...",[المنتج جه غير مطابق اللي على موقع جالي موبايل...,3.5,"EGP1,299.00",850,100+ bought,[],None,None,None
4,Bumpers,ivoler Anti-Yellow Clear Case for Samsung Gala...,https://m.media-amazon.com/images/I/715iE-RAD8...,iVoler,None,,samsung galaxy a55 5g,Clear,Polycarbonate,Classy & Unique,"[ممتاز كما بالوصف, مبدأيا المنتج لا يوجد به اي...",None,EGP120.00,110,None,"[Clear, transparent]",None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4253,Flip Cases,Case Luxury Bookstyle Clear View Window Electr...,https://m.media-amazon.com/images/I/614QWE1m8o...,Generic,None,,Samsung Galaxy S21 Ultra,Gold,"Faux Leather, Polycarbonate",★[ Fashion Design ]: Elaborately Fashion Mirro...,[هوومش طبيعي اعمل طلب ترجيع ويفضل كل ده],None,None,200,None,"[Gold, Purple, Rose Gold]",,None,Galaxy
4254,Bumpers,Bump Silicon TPU Shild Case Microfiber Lining ...,https://m.media-amazon.com/images/I/71C5d8hckU...,Generic,None,,"iPhone 14 Pro Max, 6.7 inch",Green,"Silicone, Thermoplastic Polyurethane",For Apple iphone 14 Pro Max 6.7 Inch Bump TPU ...,None,None,EGP114.00,90,None,"[Green, Purple, White, Wine Red]",,Not Water Resistant,
4255,Basic Cases,Elmo3ezz Compatible with Case for Samsung Gala...,https://m.media-amazon.com/images/I/61dHUp3mIa...,ELMO3EZZ,None,,Thermoplastic Polyurethane,None,Thermoplastic Polyurethane,1,None,None,None,110,None,[],No Embellishment,None,Galaxy
4256,Bumpers,Elmo3ezz Compatible with Case Honor X7B Slim S...,https://m.media-amazon.com/images/I/61qsAnfowF...,ELMO3EZZ,None,None,Honor X7B,None,Leather,1,None,None,None,110,None,[],,None,Solid


In [35]:
df.to_csv('cases.csv', index=False)

In [4]:
# Define the file names
files = ['smartphones.csv', 'chargers.csv', 'headphones.csv', 'cases.csv']

# Initialize an empty DataFrame for the merged result
merged_df = pd.DataFrame()

# Iterate over each file and merge them
for file in files:
    # Read the current CSV into a DataFrame
    df = pd.read_csv(file)
    
    # Perform an outer merge to combine all columns, even if they don't exist in all DataFrames
    merged_df = pd.merge(merged_df, df, how='outer') if not merged_df.empty else df

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('combined_products.csv', index=False)

print("CSV files merged successfully into 'combined_products.csv'")



CSV files merged successfully into 'combined_products.csv'


In [ ]:
browser.quit()